In [23]:
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec
import json
import numpy as np

In [3]:
model = Doc2Vec.load_word2vec_format('../datasets/google-news-word2vec/GoogleNews-vectors-negative300.bin', binary=True)

In [5]:
CNN_vect = model['CNN']

In [6]:
CNN_vect.shape

(300,)

In [9]:
para_vect = model['reports', 'bombing']

In [11]:
para_vect.shape

(2, 300)

In [60]:
type(para_vect)

numpy.ndarray

In [61]:
type(CNN_vect)

numpy.ndarray

In [12]:
sentence = LabeledSentence(words=[u'some', u'words', u'here'], labels=[u'SENT_1'])

NameError: name 'LabeledSentence' is not defined

In [13]:
def read_data(fn):
    data = []
    print "[EDEN I/O -- read_data] Files: "
    for f in fn.split(','):
        print "f: ", f
        with open("../datasets/raw-data/" + str(f) + ".json", 'rb') as infile:
            data.extend(json.load(infile))

    print "[EDEN I/O -- read_data] Data length: ", len(data)
    print "[EDEN I/O -- read_data] Data type: ", type(data)

    return json.dumps(data)

In [16]:
json_data = read_data('1,12,2,20,21,23,28,29,30,33,35,40,5,6')

[EDEN I/O -- read_data] Files: 
f:  1
f:  12
f:  2
f:  20
f:  21
f:  23
f:  28
f:  29
f:  30
f:  33
f:  35
f:  40
f:  5
f:  6
[EDEN I/O -- read_data] Data length:  2469
[EDEN I/O -- read_data] Data type:  <type 'list'>


In [124]:
def preprocess_data(data):
    # stories = [doc["_source"]["content"] for doc in data]
    # ids = [doc["_id"] for doc in data]
    
    print data[0]

    def format_entities(norm_ent):
        ents = []
        for ent in norm_ent:
            try:
                ents.append(ent['surface-form'])
            except:
                continue
        return " ".join(ents)

    d = [{"id": doc["_id"],
          "first-published": doc["_source"]["first-published"],
          "title": doc["_source"]["title"],
          "summary": doc["_source"]["title"],
          "content": doc["_source"]["content"],
          "entities": format_entities(doc["_source"]["normalised-entities"])} for doc in data]

    word2vec_signal = {}
    
    for doc in d:
        words = []
        for word in doc["content"].split(" "):
            try: 
                words.append(str(word))
            except:
                continue
        words_vecs = []
        for i, word in enumerate(words):
            try:
                words_vecs.append(model[word])
            except:
                continue
        if len(words_vecs) > 0:
            doc_vec = np.sum(words_vecs, axis = 0)
            word2vec_signal[doc["id"]] = doc_vec
        else:
            word2vec_signal[doc["id"]] = model['junk']
        

    return word2vec_signal

In [125]:
word2vec_signal = preprocess_data(data)

{u'_score': 9.824745, u'_type': u'story', u'_id': u'af3a0369-f7bb-48f7-9cac-e278b856034f', u'_index': u'stories-20150903', u'_source': {u'feed': u'NLA', u'article-count': 1, u'source-groups': [u'535e4a2e-e136-481a-9dc2-38301778dd1b', u'd6cc851e-1e0e-4988-8a31-266ad6c73d18'], u'topics': [], u'canonical-author-unanalysed': None, u'processed-time': u'2015-09-03T22:14:31.542Z', u'normalised-entities': [{u'type': u'source', u'id': u'4e8f704e-45bf-4f07-911f-b188a0c6f43f', u'name': u'The Sun'}, {u'type': u'source-group', u'id': u'535e4a2e-e136-481a-9dc2-38301778dd1b', u'name': {u'visible-to-users': [u'dec775a9-ff75-43ff-8745-32c47f383bee'], u'name': u'Content audit sources'}}, {u'surface-form': u'TESCO', u'type': u'organisation', u'id': u'Tesco', u'name': u'TESCO'}], u'title': u'CLUBCARD CUTS', u'most-recent-published': u'2015-09-03T22:13:59Z', u'content': u'TESCO credit card customers are to earn fewer Clubcard points.\r\nRewards on purchases made in other stores will drop from one point for

In [109]:
len(word2vec_signal)

2433

In [76]:
from sklearn.metrics.pairwise import cosine_similarity


In [77]:
cosine_similarity(word2vec_signal.values, word2vec_signal.values)

TypeError: float() argument must be a string or a number

In [121]:
cosine_similarity(word2vec_signal.values(),word2vec_signal.values())

array([[ 1.        ,  0.79977262,  0.49447942, ...,  0.65581739,
         0.68005711,  0.8203851 ],
       [ 0.79977262,  1.00000012,  0.61501241, ...,  0.75685763,
         0.78333962,  0.87608647],
       [ 0.49447942,  0.61501241,  0.99999994, ...,  0.53341532,
         0.62735456,  0.59910232],
       ..., 
       [ 0.65581739,  0.75685763,  0.53341532, ...,  1.        ,
         0.63225263,  0.66662061],
       [ 0.68005711,  0.78333962,  0.62735456, ...,  0.63225263,
         1.        ,  0.79419041],
       [ 0.8203851 ,  0.87608647,  0.59910232, ...,  0.66662061,
         0.79419041,  0.99999988]], dtype=float32)

In [123]:
model.similar_by_word('acquisition')

Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


KeyboardInterrupt: 

In [98]:
nf = 0
for doc in data:
    try:
        print doc["_id"]
        word2vec_signal[str(doc["_id"])]
        print "not found"
        nf +=1
    except:
        word2vec_signal[doc["id"]] = model['junk']

af3a0369-f7bb-48f7-9cac-e278b856034f
not found
d8faa5c5-4ee3-494d-940d-f16ef374c8a8
not found
5f6dea84-8493-44f0-aff6-de136a18ac9a
not found
7702887c-f5ff-48f2-88d5-3d3b7fee516f
not found
dbc63e85-dc60-4c13-9253-09ee899d7613
not found
48f1fcb7-24af-4152-92ce-a2f2b7a72e59
not found
701c10d3-cac7-4bff-aa34-a821cbd4301a
not found
096ec5dc-0f28-4a3e-8815-8897df268e85
not found
ff23d92b-67de-46c0-a8ae-55be1491d2f5
not found
cd636aef-6f35-4702-8b8f-e1cee852adee
not found
3e73d271-9a70-4fb6-877c-5e6e7d2ce675
not found
e3415aa3-057a-4c27-829a-b14b3c9e6579
not found
e560980a-1b92-4a5f-b93b-ba409a97f065
not found
34d5dda9-c4d8-447d-b984-c8b166f6ef0e
not found
8124cf73-0ff9-40ca-a745-d62de93af3e2
not found
b9dd021e-7eec-4c73-b981-7c4688dc53f4
not found
6db0e4d8-45af-48fd-b06a-5d5455b67030
not found
5ba48cc6-f0a4-45cc-a1ca-302ed9b0e12b
not found
b35c4c7a-4f77-4a7b-8fb4-4249a0609b4a
not found
9bc17970-316e-45fe-887c-121f08f49af0
not found
502e046f-2502-44bc-81b2-b9255aa396b0
not found
468e9c4e-86f8

In [99]:
nf

2469

In [92]:
len(word2vec_signal)

2433

In [ ]:
id_counter 
for doc in data

In [94]:
word2vec_signal.keys()[0]

u'0ae65f99-fdeb-45fe-8524-24c200e88eb5'

In [127]:
word2vec_signal['0ae65f99-fdeb-45fe-8524-24c200e88eb5']

array([-0.32041931,  1.43572998,  0.86917591,  2.29702759, -1.50195312,
        0.68647003,  1.92536926, -2.4967041 ,  4.43673706,  0.55953979,
       -3.92913818, -0.45800781, -0.16589355,  1.40843201, -4.04441261,
        0.69311523,  0.72638965,  3.4634285 , -0.6247406 , -0.80218506,
       -1.49008751,  2.02790833, -2.60964966,  2.21318054,  0.59648132,
       -1.141922  , -2.63696289,  4.86157227, -1.24754333, -0.75886726,
        0.44692993, -2.38056946, -4.14020538, -1.80407715,  0.7925415 ,
        0.37710571, -0.4316864 , -2.32583618,  2.18133545, -1.01306152,
        2.01074219,  0.79446411,  1.35652924,  2.87730789, -2.32775879,
       -5.93632603, -1.37213135,  1.58457947,  1.14259338,  2.69366455,
        1.58047867,  0.81506348, -1.29034424, -3.80007553, -0.50067139,
        1.76167297, -4.11297607, -2.73266602,  2.65205383, -3.8969574 ,
       -1.66661072,  0.85244751, -3.70617676, -3.01065063, -3.3431015 ,
       -0.50732422, -1.24328613,  5.06762695,  2.64715576,  3.18

In [106]:
print type(word2vec_signal.keys()[0])
print word2vec_signal.keys()[0]

<type 'unicode'>
0ae65f99-fdeb-45fe-8524-24c200e88eb5


In [104]:
print type(data[0]["_id"])
print data[0]["_id"]

<type 'unicode'>
af3a0369-f7bb-48f7-9cac-e278b856034f


In [112]:

len(set([doc["_id"] for doc in data]))

2433

In [128]:
with open('../datasets/word2vec_signal/word2vec_signal.p', 'wb') as fout:
    pickle.dump(word2vec_signal, fout, pickle.HIGHEST_PROTOCOL)

In [113]:
import pickle